# REDCap Datatype and API Export Notes

### Non-Standard Field types
- instrument_name_complete/incomplete

### Standard Field Types
- text
- radio
- dropdown
- file
- descriptive
- notes
- yesno
- checkbox


In addition to standard types, each form (instrument) has a column to indicate if form is complete/incomplete

instrument_name_complete/incomplete is exported via standard API call. 




## API Export Records (default settings)



- text
- radio
- dropdown
- file
- <s> descriptive
- notes
- yesno
- checkbox
- instrument_name_complete/incomplete


"descriptive" field type is NOT exported

# Appendix

## Pycap Methods




### project.export_fields()
    - does NOT export "file" field types
    
### project.export_metadata()
    - does NOT export "instrument_name_complete/incomplete"
    






In [1]:
import redcap

In [2]:
url = "https://qa-redcap.doh.wa.gov/api/"
api_key = "1D4B2F0ED2500C255CDEDA2F159E7725" #QA env PID75

#Create Project objects
project = redcap.Project(url, api_key)


def describe_fields(project):
    """
    create a summary dataframe to describe every standard field from default API export
    
    args:
        pycap Project object
    return:
        Dataframe
        
    """
    
    df_fields = project.export_field_names(format_type = "df")
    df_meta = project.export_metadata(format_type = "df")

    #drop "record_id" field, it is the row index of dataframe export 
    #IS THIS STANDARD FIELD?? Possible bug if record_id is renamed?
    df_meta = df_meta.drop("record_id")
    df_fields = df_fields.drop("record_id")

    #only need 2 fields from metadata export
    df_meta = df_meta[["form_name", "field_type"]].copy()

    #combine fields from metadata and from export_field_names 
    df_fields_2 = df_fields.join(df_meta)
    
    #add field names that have type "file" (from metadata)
    files_df = df_meta[df_meta["field_type"] == "file"].copy()
    df_fields_3 = pd.concat([df_fields_2, files_df])
    
    return df_fields_3

In [ ]:
df = describe_fields(project)
df

In [4]:
df.shape

(471, 4)

# Development

# Appendix